<a href="https://colab.research.google.com/github/valeriamincyt/valeriamincyt-openalex_inst_pars_fer/blob/main/002_Making_Model_Files_Fer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import json
import json,urllib.request
!pip install redshift_connector ##no instalar en script
import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import re
import os
!pip install unidecode
from unidecode import unidecode
from collections import Counter
from math import ceil
!pip install langdetect
from langdetect import detect
from random import sample

from google.colab import drive

!pip install pyalex
import pyalex
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
from pyalex import config

!pip install colab-convert ##no instalar en script


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=47971849ffa143f4b51006bb10301f08b698ee78fcf08ef4d6b78c3266b6f794
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
  Preparing metadata (

## Support Files

Throughout the modeling process, some of the model artifacts needed to be updated and so this notebook was used to quickly update those files.

In [ ]:
## comentar siguiente línea en script
drive.mount('/content/drive')

Mounted at /content/drive


**Cambiar path**

In [ ]:
base_path = '/content/drive/MyDrive/openalex-institution-parsing/'
# location where current files are located
curr_model_artifacts_location = os.path.join(base_path, 'Crudos/institution_tagger_v2_artifacts/')

In [ ]:
# Load the needed files
with open(f"{curr_model_artifacts_location}departments_list.pkl", "rb") as f:
    departments_list = pickle.load(f)
print("Loaded list of departments")

with open(f"{curr_model_artifacts_location}countries_list_flat.pkl", "rb") as f:
    countries_list_flat = pickle.load(f)
print("Loaded flat list of countries")

with open(f"{curr_model_artifacts_location}countries.json", "r") as f:
    countries_dict = json.load(f)
print("Loaded countries dictionary")

with open(f"{curr_model_artifacts_location}city_country_list.pkl", "rb") as f:
    city_country_list = pickle.load(f)
print("Loaded strings of city/country combinations")

Loaded list of departments
Loaded flat list of countries
Loaded countries dictionary
Loaded strings of city/country combinations


### Looking at ROR

In [ ]:
def get_geoname_admin(address_dict):
    try:
        geoname_admin = address_dict['geonames_city']['geonames_admin1']['name']
    except:
        geoname_admin = "None"

    return geoname_admin

In [ ]:
def get_final_region(ror_state, ror_region):
    if isinstance(ror_state, str):
        return ror_state
    elif isinstance(ror_region, str):
        return ror_region
    else:
        return None

In [ ]:
def check_for_backwards_univ(curr_names):
    names = curr_names.copy()
    for one_name in curr_names:
        split_name = one_name.split(" ")
        if len(split_name) == 3:
            if (split_name[0] == 'University') & (split_name[1] == 'of'):
                names.append(f"{split_name[2]} University")
        elif len(split_name) == 2:
            if (split_name[1] == 'University'):
                names.append(f"University of {split_name[0]}")
        else:
            pass
    return names

In [ ]:
def add_names_to_list(all_names):
    names = all_names.copy()
    if "Harvard University" in names:
        names.append("Harvard Medical School")
    elif "University of Oxford" in names:
        names.append("Oxford University")
    else:
        pass

    return names

In [ ]:
def get_exact_names(name, aliases, acronyms, labels):
    all_names = [name] + aliases + acronyms + [i['label'] for i in labels]
    all_names = add_names_to_list(all_names)
    all_names = [x for x in all_names if ~x.startswith('Department of')]
    all_names_clean = [string_match_clean(x) for x in all_names]
    return [x for x in all_names_clean if len(x) > 4]

In [ ]:
def string_match_clean(text):
    #replace "&" with "and"
    if "r&d" not in text.lower():
        text = text.replace(" & ", " and ")

    # take country out
    if text.strip().endswith(")"):
        for country in countries_list_flat:
            if text.strip().endswith(f"({country})"):
                text = text.replace(f"({country})", "")

    # use unidecode
    text = unidecode(text.strip())

    # replacing common abbreviations
    text = text.replace("Univ.", "University")
    text = text.replace("Lab.", "Laboratory")

    # take out spaces, commas, dashes, periods, etcs
    text = re.sub("[^0-9a-zA-Z]", "", text)

    return text

In [ ]:
def list_of_all_names(oa_name, ror_names, extra_names, use_extra_names=False):
    banned_names = ['UniversityHospital','Coastal','Brunswick','Continental']
    if isinstance(ror_names, list):
        pass
    else:
        ror_names = []

    if isinstance(oa_name, str):
        oa_string = [string_match_clean(oa_name)]
    else:
        oa_string = []

    if use_extra_names:
        if isinstance(extra_names, list):
            pass
        else:
            extra_names = []
    else:
        extra_names = []

    return [x for x in list(set(oa_string+ror_names+extra_names)) if
            ((len(x) > 4) & (x not in banned_names))]

In [ ]:
### ALTERNATIVA DE OA_static_institutions_single_file.parquet bajando desde Python

pyalex.config.email = "mfartola.mincyt@gmail.com"

config.max_retries = 0
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

In [ ]:
data = urllib.request.urlopen("https://api.openalex.org/institutions?filter=country_code:AR&per-page=200&page=1").read()
data1 = urllib.request.urlopen("https://api.openalex.org/institutions?filter=country_code:AR&per-page=200&page=2").read()

output = json.loads(data)
output1 = json.loads(data1)

argentina1=pd.DataFrame(output['results'])
argentina2=pd.DataFrame(output1['results'])
argentina=pd.concat([argentina1,argentina2])
argentina=pd.DataFrame(argentina)
print(argentina.head())
argentina.shape
institutions_df=argentina
institutions_df.columns

                                id                        ror  \
0  https://openalex.org/I151201029  https://ror.org/03cqe8w59   
1   https://openalex.org/I24354313  https://ror.org/0081fs513   
2  https://openalex.org/I874386039  https://ror.org/01tjs6929   
3  https://openalex.org/I166401450  https://ror.org/056tb7j80   
4   https://openalex.org/I53241121  https://ror.org/05rxmkq09   

                                                 display_name country_code  \
0  Consejo Nacional de Investigaciones Científicas y Técnicas           AR   
1                                  University of Buenos Aires           AR   
2                            Universidad Nacional de La Plata           AR   
3                             Universidad Nacional de Córdoba           AR   
4                      Fundación Ciencias Exactas y Naturales           AR   

         type                                            type_id  \
0  government  https://openalex.org/institution-types/government   
1   

Index(['id', 'ror', 'display_name', 'country_code', 'type', 'type_id',
       'lineage', 'homepage_url', 'image_url', 'image_thumbnail_url',
       'display_name_acronyms', 'display_name_alternatives', 'repositories',
       'works_count', 'cited_by_count', 'summary_stats', 'ids', 'geo',
       'international', 'associated_institutions', 'counts_by_year', 'roles',
       'topics', 'topic_share', 'x_concepts', 'is_super_system',
       'works_api_url', 'updated_date', 'created_date'],
      dtype='object')

In [ ]:
institutions_df['affiliation_id'] = institutions_df['id'].apply(lambda x: x.split("/")[-1])
institutions_df['affiliation_id']=institutions_df['affiliation_id'].apply(lambda x: x.split("I")[-1])

institutions_df['ror_id'] = institutions_df['ror'].apply(lambda x: x.split("/")[-1])

print(institutions_df.head(1))

                                id                        ror  \
0  https://openalex.org/I151201029  https://ror.org/03cqe8w59   

                                                 display_name country_code  \
0  Consejo Nacional de Investigaciones Científicas y Técnicas           AR   

         type                                            type_id  \
0  government  https://openalex.org/institution-types/government   

                                                                                                 lineage  \
0  [https://openalex.org/I151201029, https://openalex.org/I4210123736, https://openalex.org/I4387155568]   

                homepage_url  \
0  http://www.conicet.gov.ar   

                                                                                                 image_url  \
0  https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/Conicet%20Logo%20con%20letras.png   

                                                                        

In [ ]:
institutions_df.columns

Index(['id', 'ror', 'display_name', 'country_code', 'type', 'type_id',
       'lineage', 'homepage_url', 'image_url', 'image_thumbnail_url',
       'display_name_acronyms', 'display_name_alternatives', 'repositories',
       'works_count', 'cited_by_count', 'summary_stats', 'ids', 'geo',
       'international', 'associated_institutions', 'counts_by_year', 'roles',
       'topics', 'topic_share', 'x_concepts', 'is_super_system',
       'works_api_url', 'updated_date', 'created_date', 'affiliation_id',
       'ror_id'],
      dtype='object')

In [ ]:
institutions_df=institutions_df[['affiliation_id','display_name','ror_id']]
print(institutions_df.head(2))

  affiliation_id                                                display_name  \
0      151201029  Consejo Nacional de Investigaciones Científicas y Técnicas   
1       24354313                                  University of Buenos Aires   

      ror_id  
0  03cqe8w59  
1  0081fs513  


In [ ]:
##Carga de ror

file_path = os.path.join(base_path, 'Crudos/v1.53-2024-10-14-ror-data.json')

ror = pd.read_json(file_path)
print(ror.head(2))
print(ror.columns)

                          id                 name        types  \
0  https://ror.org/04ttjf776      RMIT University  [Education]   
1  https://ror.org/01rxfrp27  La Trobe University  [Education]   

                          links  \
0    [https://www.rmit.edu.au/]   
1  [http://www.latrobe.edu.au/]   

                                                aliases acronyms  status  \
0  [Royal Melbourne Institute of Technology University]   [RMIT]  active   
1                                                    []       []  active   

                                      wikipedia_url labels email_address  \
0      http://en.wikipedia.org/wiki/RMIT_University     []          None   
1  http://en.wikipedia.org/wiki/La_Trobe_University     []          None   

  ip_addresses  established  \
0           []       1887.0   
1           []       1964.0   

                                               country  \
0  {'country_code': 'AU', 'country_name': 'Australia'}   
1  {'country_code': 'AU', '

In [ ]:
ror['address'] = ror['addresses'].apply(lambda x: x[0])
ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
ror['ror_id']=ror['ror_id'].apply(lambda x: x.split("I")[-1])
ror['types'] = ror['types'].apply(lambda x: x[0] if len(x)>0 else "")

In [ ]:
ror['country_code'] = ror['country'].apply(lambda x: x['country_code'])
ror['country_name'] = ror['country'].apply(lambda x: x['country_name'])
ror['city'] = ror['address'].apply(lambda x: x['city'])
ror['state'] = ror['address'].apply(lambda x: x['state'])
ror['region'] = ror['address'].apply(get_geoname_admin)

In [ ]:
ror.columns

Index(['id', 'name', 'types', 'links', 'aliases', 'acronyms', 'status',
       'wikipedia_url', 'labels', 'email_address', 'ip_addresses',
       'established', 'country', 'relationships', 'addresses', 'external_ids',
       'address', 'ror_id', 'country_code', 'country_name', 'city', 'state',
       'region'],
      dtype='object')

In [ ]:
ror_to_join = ror[['ror_id','name','status','types','aliases','acronyms','labels','city',
                   'state','region','country_name']].copy()

ror_to_join.columns = ['ror_id','name','status','types','aliases','acronyms','labels','city',
                       'temp_state','temp_region','country']

In [ ]:
ror_to_join['state'] = ror_to_join.apply(lambda x: get_final_region(x.temp_state, x.temp_region), axis=1)
print(ror_to_join.head(1))

      ror_id             name  status      types  \
0  04ttjf776  RMIT University  active  Education   

                                                aliases acronyms labels  \
0  [Royal Melbourne Institute of Technology University]   [RMIT]     []   

        city temp_state temp_region    country     state  
0  Melbourne   Victoria    Victoria  Australia  Victoria  


In [ ]:
inst_ror = ror_to_join.merge(institutions_df[['ror_id','affiliation_id','display_name']],how='inner', on='ror_id')
inst_ror.shape

(331, 14)

In [ ]:
print(inst_ror.head(n=3))

      ror_id                                 name  status      types aliases  \
0  00pt8r998  National University of the Littoral  active  Education      []   
1  00htwgm11         National University of Salta  active  Education      []   
2  01yjy8p80         National University of Luján  active  Education      []   

  acronyms  \
0    [UNL]   
1   [UNSa]   
2       []   

                                                                                                                           labels  \
0  [{'label': 'Universidad Nacional del Litoral', 'iso639': 'es'}, {'label': 'Université Nationale du Littoral', 'iso639': 'fr'}]   
1                                                                    [{'label': 'Universidad Nacional de Salta', 'iso639': 'es'}]   
2                                                                    [{'label': 'Universidad Nacional de Luján', 'iso639': 'es'}]   

       city temp_state   temp_region    country         state affiliation_id  \
0  Santa 

#### Getting file of multi-institution names

In [ ]:
multi_inst_names_df = ror_to_join[['ror_id','name']].merge(institutions_df[['ror_id','affiliation_id']],
                                                        how='left', on='ror_id') \
['name'].value_counts().reset_index()

In [ ]:
print(multi_inst_names_df.head(25))

                                                           name  count
0                                            Ministry of Health     53
1                                         Ministry of Education     23
2                                             Argosy University     19
3                                           Ministry of Justice     16
4                                   Ministry of Foreign Affairs     16
5                                           Ministry of Culture     13
6                                    Government Medical College     13
7                                           St. Luke's Hospital     12
8               International Institute of Tropical Agriculture     11
9                                           Ministry of Finance     11
10                                     Médecins Sans Frontières     10
11                                           St Mary's Hospital     10
12                                          Ministry of Defence     10
13    

In [ ]:
multi_inst_names = multi_inst_names_df[multi_inst_names_df['count']>1]
multi_inst_names
multi_inst_names=multi_inst_names['name'].tolist()
#multi_inst_names

In [ ]:
## paso a entero a los ids antes de pasarlo a lista
multi_inst_names_ids = inst_ror[inst_ror['name'].isin(multi_inst_names)]['affiliation_id'].astype(int).tolist()

In [ ]:
with open(f"{curr_model_artifacts_location}multi_inst_names_ids1.pkl", "wb") as f:
    pickle.dump(multi_inst_names_ids, f)

In [ ]:
multi_inst_names_ids

[4210139084,
 4210110275,
 4210098121,
 1333839041,
 4210158077,
 4210123859,
 4210123736]

### Getting Mapping of Inactive Institutions

There are institutions in ROR that are listed as "Withdrawn" or "Inactive". There was some thought to use the old data associated with these ROR IDs and apply them to successors but for this model, we decided to hold off on doing this because we were unsure if there would be a benefit to doing so. Therefore, the code is provided but this data was not used in building the model.

In [ ]:
def get_successors_from_relationships(relationships):
    successors = []
    parents = []
    for relationship in relationships:
        if relationship['type'] == 'Successor':
            successors.append(relationship['id'].split("/")[-1])
        elif relationship['type'] == 'Parent':
            parents.append(relationship['id'].split("/")[-1])
        else:
            pass
    return [successors, parents]

In [ ]:
def get_extra_names(ror_id):
    if ror_id in successor_dict.keys():
        extra_names = []
        for old_id in successor_dict[ror_id]['ror_id']:
            extra_names += old_name_data[old_id]['successor_names']

        extra_names = list(set(extra_names))
    else:
        extra_names = []

    return extra_names

In [ ]:
withdrawn_or_inactive_df = ror[ror['status'].isin(['withdrawn','inactive'])].copy()
withdrawn_or_inactive_df.shape

(1973, 23)

In [ ]:
withdrawn_or_inactive_df['successors'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[0])
withdrawn_or_inactive_df['parents'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[1])

KeyError: 'successors_parents'

In [ ]:
withdrawn_or_inactive_df['successor_len'] = withdrawn_or_inactive_df['successors'].apply(len)

In [ ]:
to_add_to_successors = withdrawn_or_inactive_df[withdrawn_or_inactive_df['successor_len']==1].copy()
to_add_to_successors['successor'] = to_add_to_successors['successors'].apply(lambda x: x[0])
to_add_to_successors.shape

In [ ]:
to_add_to_successors['successor_names'] = to_add_to_successors.apply(lambda x: get_exact_names(x['name'],
                                                                                               x.aliases,
                                                                                               x.acronyms,
                                                                                               x.labels),
                                                                     axis=1)

In [ ]:
old_name_data = to_add_to_successors.set_index('ror_id')[['successor_names']].to_dict(orient='index')

In [ ]:
successor_dict = to_add_to_successors.groupby('successor')['ror_id'].apply(list).reset_index()\
    .set_index('successor').to_dict(orient='index')

In [ ]:
## para agregar columna "parent" al ror
ror['successors_parents'] = ror['relationships'].apply(get_successors_from_relationships)
ror['parents'] = ror['successors_parents'].apply(lambda x: x[1])

# Convert 'parents' column to string before creating DataFrame
# ----> Convert list to string by joining with a delimiter (e.g., ',')
relationships = pd.DataFrame(ror['parents'].apply(lambda x: ','.join(x) if isinstance(x, list) else x))
print(relationships)
print(relationships.shape)

ror_corto = pd.DataFrame(ror[['id', 'name']])
ror_corto['id'] = ror_corto['id'].apply(lambda x: x.split("/")[-1])

# Ensure 'id' column in ror_corto is also of string type if it contains lists
ror_corto['id'] = ror_corto['id'].astype(str)

# Merge using the string representation of 'parents'
ror['parent_str']=relationships.merge(ror_corto, left_on="parents", right_on="id",how='left')['name']
ror.head(5)

inst_ror['parent_str']=ror['parent_str']

inst_ror['parent_str']=inst_ror['parent_str'].fillna('')
print(inst_ror.head(3))

### Getting ROR String Matching File and Affiliation Dictionary

In [ ]:
inst_ror['extra_names'] = inst_ror['ror_id'].apply(get_extra_names)
inst_ror['extra_names']


In [ ]:
inst_ror['name_cparent']=inst_ror['name'].astype(str)+" "+inst_ror['parent_str'].astype(str)
inst_ror['name','name_cparent'].head(15)

In [ ]:
inst_ror['exact_names'] = (inst_ror.apply(lambda x: get_exact_names(x['name_cparent'], x.aliases,###cambie name por name_cparent
                                                                         x.acronyms, x.labels), axis=1))

##cómo se arma exact names?
inst_ror.apply(lambda x: get_exact_names(x['name'], x.aliases,x.acronyms, x.labels), axis=1)

In [ ]:
inst_ror['final_names'] = inst_ror.apply(lambda x: list_of_all_names(x.display_name, x.exact_names,
                                                                     x.extra_names,
                                                                     use_extra_names=False), axis=1)
##cómo se arma final_names
inst_ror.apply(lambda x: list_of_all_names(x.display_name, x.exact_names,
                                                                     x.extra_names,
                                                                     use_extra_names=False), axis=1)

In [ ]:
## lo paso a entero antes de hacerlo indice
inst_ror['affiliation_id']=inst_ror['affiliation_id'].astype(int)

In [ ]:
new_affiliation_dict = inst_ror.set_index('affiliation_id')[['display_name','city','state',
                                                             'country','final_names','ror_id','types']] \
.to_dict(orient='index')

In [ ]:
new_affiliation_dict

In [ ]:
with open(f"{curr_model_artifacts_location}full_affiliation_dict1.pkl", "wb") as f:
    pickle.dump(new_affiliation_dict, f)

In [ ]:
new_affiliation_dict

### Updating the city/country file

This file is used to check the affiliation string to make sure it doesn't exactly match up with a city/region/country combo with no additional information.

In [ ]:
city_region_country = inst_ror.drop_duplicates(subset=['city','country']).copy()
city_region_country.shape

In [ ]:
new_city_country_list = list(set([f"{i}{j}" for i,j in zip(city_region_country['city'].tolist(),
                                   city_region_country['country'].tolist())] +
         [f"{i}{j}{k}"for i,j,k in zip(city_region_country['city'].tolist(),
                                             city_region_country['state'].tolist(),
                                             city_region_country['country'].tolist()) if j ] +
         [f"{i}{j}" for i,j in zip(city_region_country['state'].tolist(),
                                   city_region_country['country'].tolist()) if i] +
         [f"{i}" for i in city_region_country['country'].tolist()] +
         [f"{i}" for i in city_region_country['state'].tolist() if i]))

new_city_country_list = list(set([string_match_clean(x) for x in new_city_country_list]))

In [ ]:
len(new_city_country_list)

In [ ]:
with open(f"{curr_model_artifacts_location}city_country_list1.pkl", "wb") as f:
    pickle.dump(new_city_country_list, f)

### Flat country file is up to date

Flat country file is needed to search for country in the string for the model.

In [ ]:
len(list(set(countries_list_flat)))

In [ ]:
all_countries = []
for i in countries_dict.values():
    all_countries += i

In [ ]:
len(list(set(all_countries)))

In [ ]:
with open(f"{curr_model_artifacts_location}countries_list_flat1.pkl", "wb") as f:
    pickle.dump(list(set(all_countries)), f)

### Departments list update

Takes the old department list and updates it with additional department names.

In [ ]:
file_path = os.path.join(base_path, 'V2/001_Exploration/ror_string_beginnings/Education_dept')
with open(file_path, 'r') as f:
    education_dept_begs = f.readlines()

education_dept_begs = list(set([x.rstrip('\n') for x in education_dept_begs]))

In [ ]:
departments_list = ['Psychology','Nephrology','Other departments','Other Departments','Nursing & Midwifery',
                    'Literature and Creative Writing','Neuroscience','Engineering','Computer Science',
                    'Chemistry','Biology','Medicine']

In [ ]:
new_departments_list = list(set(departments_list + education_dept_begs))

In [ ]:
with open(f"{curr_model_artifacts_location}departments_list1.pkl", "wb") as f:
    pickle.dump(new_departments_list, f)

### Make affiliation IDs integers

In [ ]:
with open(f"{curr_model_artifacts_location}affiliation_vocab.pkl", "rb") as f:
    affiliation_vocab_basic = pickle.load(f)

print(affiliation_vocab_basic)
print("Loaded affiliation vocab basic")


new_affiliation_vocab_basic = {int(i):int(j) for j,i in affiliation_vocab_basic.items()}
print(new_affiliation_vocab_basic)
print("New vocab basic")


In [ ]:
with open(f"{curr_model_artifacts_location}new_affiliation_vocab1.pkl", "wb") as f:
    pickle.dump(new_affiliation_vocab_basic, f)

In [ ]:
print(new_affiliation_vocab_basic)

In [ ]:
##comentar siguiente línea en script
###!colab-convert /content/drive/MyDrive/openalex-institution-parsing/V2/001_Exploration/Notebooks/002_Making_Model_Files_Fer.ipynb /content/drive/MyDrive/openalex-institution-parsing/V2/001_Exploration/Notebooks/002_Making_Model_Files_Fer.py